In [ ]:
!nvidia-smi

Mon Jun 16 15:32:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("/content/drive/MyDrive/Myproject/yolo11m.pt")

In [ ]:
res = model.train(cfg = "/content/drive/MyDrive/Myproject/args_detect_2_classes.yaml",
                  project = "/content/drive/MyDrive/Myproject/weight_v1",)

In [ ]:
import pandas as pd

csv_path = "/content/drive/MyDrive/Myproject/weight_v1/train/results.csv"  # chỉnh đúng path thực tế
df = pd.read_csv(csv_path)
print(df.tail())  # Hiện các chỉ số các epoch cuối cùng


    epoch     time  train/box_loss  train/cls_loss  train/dfl_loss  \
65     66  4896.19         0.70400         0.35284         0.97190   
66     67  4968.35         0.70128         0.34632         0.96782   
67     68  5040.89         0.69826         0.34953         0.96904   
68     69  5113.10         0.68841         0.34269         0.96198   
69     70  5184.89         0.67987         0.34037         0.95801   

    metrics/precision(B)  metrics/recall(B)  metrics/mAP50(B)  \
65               0.96648            0.96550           0.98320   
66               0.96935            0.96320           0.98358   
67               0.95690            0.96965           0.98474   
68               0.96017            0.96522           0.98298   
69               0.96393            0.95804           0.98305   

    metrics/mAP50-95(B)  val/box_loss  val/cls_loss  val/dfl_loss    lr/pg0  \
65              0.71924       1.00485       0.42955       1.14049  0.000121   
66              0.72309       

In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load các mô hình
person_model = YOLO("yolov8n.pt")  # phát hiện người
lanyard_model = YOLO("/content/drive/MyDrive/Myproject/weight_v1/train/weights/best.pt")  # phát hiện thẻ

# Đường dẫn video input và output
input_video = "/content/ TEST!.mp4"
output_video = "/content/output1_result.mp4"

# Mở video input
cap = cv2.VideoCapture(input_video)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Tạo video writer để lưu kết quả
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect người
    person_results = person_model(frame, conf=0.5)
    people_boxes = person_results[0].boxes

    for box in people_boxes:
        if int(box.cls[0]) == 0 and float(box.conf[0]) > 0.5:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            crop = frame[y1:y2, x1:x2]

            # Bỏ qua nếu crop quá nhỏ
            if crop.shape[0] < 30 or crop.shape[1] < 30:
                continue

            # Detect thẻ trong crop
            result = lanyard_model(crop, conf=0.4)[0]

            for bbox in result.boxes:
                cx1, cy1, cx2, cy2 = map(int, bbox.xyxy[0].tolist())
                class_id = int(bbox.cls[0])
                label = result.names[class_id]
                conf = float(bbox.conf[0])

                # Chuyển tọa độ bbox từ crop về gốc frame
                abs_x1, abs_y1 = x1 + cx1, y1 + cy1
                abs_x2, abs_y2 = x1 + cx2, y1 + cy2

                # Vẽ bbox thẻ
                cv2.rectangle(frame, (abs_x1, abs_y1), (abs_x2, abs_y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{label} {conf:.2f}', (abs_x1, abs_y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # Vẽ bbox người
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, 'Person', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Ghi frame kết quả vào video output
    out.write(frame)
    frame_count += 1
    if frame_count % 10 == 0:
        print(f"✅ Processed {frame_count} frames...")

# Hoàn tất
cap.release()
out.release()
print("🎥 Video đã xử lý xong và lưu tại:", output_video)


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install packages
!pip install ultralytics opencv-python

# Import libraries
import os
import cv2
import numpy as np
from ultralytics import YOLO

# Prepare output folder
output_dir = '/content/people_wear'
os.makedirs(output_dir, exist_ok=True)

# Load YOLO models
model_person = YOLO('yolov8n.pt')
model_badge = YOLO('/content/drive/MyDrive/Myproject/weight_v1/train/weights/best.pt')

# Video path
video_path = '/content/V.mp4'
cap = cv2.VideoCapture(video_path)
frame_count = 0

# Processing
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame_count += 1
    results_person = model_person(frame, classes=[0], verbose=False)[0]

    for box_person in results_person.boxes.xyxy.cpu().numpy():
        x1, y1, x2, y2 = map(int, box_person)
        person_crop = frame[y1:y2, x1:x2]

        results_badge = model_badge(person_crop, verbose=False)[0]

        person_h, person_w = person_crop.shape[:2]
        center_person = np.array([person_w / 2, person_h / 2])

        wear_badge = False
        for box_badge in results_badge.boxes.xyxy.cpu().numpy():
            bx1, by1, bx2, by2 = map(int, box_badge)
            badge_center = np.array([(bx1 + bx2) / 2, (by1 + by2) / 2])

            distance = np.linalg.norm(badge_center - center_person)
            threshold_distance = 0.3 * min(person_w, person_h)

            if distance <= threshold_distance:
                wear_badge = True
                break

        if wear_badge:
            save_path = os.path.join(output_dir, f'frame{frame_count}_person.jpg')
            cv2.imwrite(save_path, person_crop)
            print(f"Saved: {save_path}")

cap.release()
print("Inference completed.")

In [ ]:
# Cài đặt thư viện nếu chưa có
!pip install -q ultralytics opencv-python

from ultralytics import YOLO
import cv2
import os
import numpy as np

# Load mô hình YOLO
model_person = YOLO('yolov8n.pt')  # Dùng YOLO nhẹ để detect người
model_badge = YOLO('/content/drive/MyDrive/Myproject/weight_v1/train/weights/best.pt')  # Model phát hiện dây/thẻ

# Tạo thư mục lưu ảnh kết quả
output_dir = '/content/drive/MyDrive/Myproject/people_wear'
os.makedirs(output_dir, exist_ok=True)

# Đọc video đầu vào
video_path = '/content/TEST5.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Dictionary lưu kết quả tốt nhất cho mỗi người
best_person_shots = {}

frame_count = 0
print("🚀 Bắt đầu xử lý video...\n")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect & track người
    result_person = model_person.track(frame, persist=True, conf=0.5, classes=[0])[0]

    for box in result_person.boxes:
        if box.id is None:
            continue

        person_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        w, h = x2 - x1, y2 - y1
        person_crop = frame[y1:y2, x1:x2]

        # Chạy model badge trên ảnh crop người
        badge_result = model_badge(person_crop, conf=0.3)[0]

        best_conf = 0
        has_badge = False

        for badge_box in badge_result.boxes:
            bx1, by1, bx2, by2 = map(int, badge_box.xyxy[0])
            bw, bh = bx2 - bx1, by2 - by1
            bc_x = bx1 + bw // 2
            bc_y = by1 + bh // 2

            cx = w // 2
            cy = h // 2
            radius = int(min(w, h) * 0.3)

            dist = np.sqrt((bc_x - cx)**2 + (bc_y - cy)**2)

            if dist <= radius:
                has_badge = True
                badge_conf = float(badge_box.conf[0])
                best_conf = max(best_conf, badge_conf)

        if has_badge:
            current_best = best_person_shots.get(person_id, {'conf': 0.0})
            if best_conf > current_best['conf']:
                best_person_shots[person_id] = {
                    'conf': best_conf,
                    'crop': person_crop.copy()
                }

    frame_count += 1
    print(f"🟢 Đã xử lý {frame_count}/{total_frames} frames", end='\r')

cap.release()

# Sau khi xong → lưu mỗi người 1 ảnh
for pid, data in best_person_shots.items():
    filename = f"{output_dir}/person_{pid}_best.jpg"
    cv2.imwrite(filename, data['crop'])

print(f"\n✅ Hoàn tất! Đã lưu {len(best_person_shots)} người đeo thẻ với confidence cao nhất.")


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.

0: 640x256 (no detections), 21.2ms
Speed: 2.1ms preprocess, 21.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 256)

0: 640x256 (no detections), 19.6ms
Speed: 2.0ms preprocess, 19.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 256)
🟢 Đã xử lý 284/821 frames
0: 352x640 2 persons, 11.8ms
Speed: 3.6ms preprocess, 11.8ms inference, 2.3ms postprocess per image at shape (1, 3, 352, 640)

0: 640x256 (no detections), 20.7ms
Speed: 2.1ms preprocess, 20.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 256)

0: 640x256 (no detections), 15.4ms
Speed: 2.3ms preprocess, 15.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 256)
🟢 Đã xử lý 285/821 frames
0: 352x640 2 persons, 11.3ms
Speed: 3.6ms preprocess, 11.3ms inference, 2.3ms postprocess per image at shape (1, 3, 352, 640)

0: 640x224 (no detections), 18.8ms
Speed: 2.0ms preprocess, 18.8ms inference, 0.8ms postprocess per image 

In [ ]:
import cv2
from ultralytics import YOLO
import os
import numpy as np

# Load models
model_person = YOLO("yolov8n.pt")  # người
model_badge = YOLO("/content/drive/MyDrive/Myproject/weight_v1/train/weights/best.pt")  # thẻ

# Output setup
output_dir = "/content/drive/MyDrive/Myproject/people_wear"
os.makedirs(output_dir, exist_ok=True)

# Input & Output video
video_path = "/content/drive/MyDrive/Myproject/ TEST!.mp4"
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

video_out_path = "/content/drive/MyDrive/Myproject/output1_detect_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_video = cv2.VideoWriter(video_out_path, fourcc, fps, (width, height))

# State
frame_count = 0
saved_ids = set()

# Process video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results_person = model_person.track(frame, persist=True, conf=0.5, classes=[0], tracker="botsort.yaml")[0]

    for box in results_person.boxes:
        if box.id is None:
            continue

        person_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        w, h = x2 - x1, y2 - y1
        cx, cy = w // 2, h // 2
        radius = int(0.3 * min(w, h))

        person_crop = frame[y1:y2, x1:x2]
        badge_results = model_badge(person_crop, conf=0.5)[0]

        has_strap = False
        has_badge = False
        center_match = False

        for b in badge_results.boxes:
            cls_id = int(b.cls[0])
            conf = float(b.conf[0])
            bx1, by1, bx2, by2 = map(int, b.xyxy[0])
            bw, bh = bx2 - bx1, by2 - by1
            bc_x = bx1 + bw // 2
            bc_y = by1 + bh // 2
            dist = np.sqrt((bc_x - cx) ** 2 + (bc_y - cy) ** 2)
            if dist <= radius:
                center_match = True
            if cls_id == 0:
                has_strap = True
            elif cls_id == 1:
                has_badge = True

        if (has_strap or has_badge) and center_match:
            label = f"Person {person_id} (with badge)"
            color = (0, 255, 0)
            if person_id not in saved_ids:
                saved_ids.add(person_id)
                cv2.imwrite(f"{output_dir}/person_{person_id}.jpg", person_crop)
        else:
            label = f"Person {person_id}"
            color = (0, 0, 255)

        # Vẽ bounding box lên frame gốc
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out_video.write(frame)
    frame_count += 1
    print(f"🟢 Đã xử lý {frame_count}/{total_frames} frames", end="\r")

cap.release()
out_video.release()
print(f"\n✅ Video output lưu tại: {video_out_path}")
print(f"📸 Đã lưu {len(saved_ids)} người có đeo thẻ vào: {output_dir}")


Streaming output truncated to the last 5000 lines.

0: 640x288 (no detections), 581.9ms
Speed: 3.4ms preprocess, 581.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 288)
🟢 Đã xử lý 2844/8247 frames
0: 352x640 1 person, 132.6ms
Speed: 5.3ms preprocess, 132.6ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 640)

0: 640x320 (no detections), 663.9ms
Speed: 3.4ms preprocess, 663.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 320)
🟢 Đã xử lý 2845/8247 frames
0: 352x640 1 person, 133.2ms
Speed: 3.8ms preprocess, 133.2ms inference, 1.2ms postprocess per image at shape (1, 3, 352, 640)

0: 640x320 (no detections), 667.7ms
Speed: 3.3ms preprocess, 667.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 320)
🟢 Đã xử lý 2846/8247 frames
0: 352x640 1 person, 127.3ms
Speed: 3.7ms preprocess, 127.3ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 640)

0: 640x288 (no detections), 608.8ms
Speed: 4.1ms preprocess, 608.8ms inference

In [ ]:
import cv2
from ultralytics import YOLO
import os
import numpy as np

# Load models
model_person = YOLO("yolov8n.pt")
model_badge = YOLO("/content/drive/MyDrive/Myproject/weight_v1/train/weights/best.pt")

# Output folders
output_dir_card = "/content/drive/MyDrive/Myproject/people_wear1"
output_dir_no_card = "/content/drive/MyDrive/Myproject/person_no_card1"
os.makedirs(output_dir_card, exist_ok=True)
os.makedirs(output_dir_no_card, exist_ok=True)

# Input & Output video
video_path = "/content/drive/MyDrive/Myproject/ TEST!.mp4"
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

video_out_path = "/content/drive/MyDrive/Myproject/output1_detect_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_video = cv2.VideoWriter(video_out_path, fourcc, fps, (width, height))

# State
frame_count = 0
saved_ids_card = set()      # Người có thẻ
saved_ids_no_card = set()   # Người không có thẻ

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results_person = model_person.track(frame, persist=True, conf=0.5, classes=[0], tracker="botsort.yaml")[0]

    for box in results_person.boxes:
        if box.id is None:
            continue

        person_id = int(box.id[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        w, h = x2 - x1, y2 - y1
        cx, cy = w // 2, h // 2
        radius = int(0.3 * min(w, h))

        person_crop = frame[y1:y2, x1:x2]
        badge_results = model_badge(person_crop, conf=0.5)[0]

        has_strap = False
        has_badge = False
        center_match = False

        for b in badge_results.boxes:
            cls_id = int(b.cls[0])
            conf = float(b.conf[0])
            bx1, by1, bx2, by2 = map(int, b.xyxy[0])
            bw, bh = bx2 - bx1, by2 - by1
            bc_x = bx1 + bw // 2
            bc_y = by1 + bh // 2
            dist = np.sqrt((bc_x - cx) ** 2 + (bc_y - cy) ** 2)
            if dist <= radius:
                center_match = True
            if cls_id == 0:
                has_strap = True
            elif cls_id == 1:
                has_badge = True

        if (has_strap or has_badge) and center_match:
            label = f"Person {person_id} (with badge)"
            color = (0, 255, 0)
            if person_id not in saved_ids_card:
                saved_ids_card.add(person_id)
                cv2.imwrite(f"{output_dir_card}/person_{person_id}.jpg", person_crop)
        else:
            label = f"Person {person_id}"
            color = (0, 0, 255)
            if (person_id not in saved_ids_no_card) and (person_id not in saved_ids_card):
                saved_ids_no_card.add(person_id)
                cv2.imwrite(f"{output_dir_no_card}/person_{person_id}.jpg", person_crop)

        # Vẽ bounding box lên frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out_video.write(frame)
    frame_count += 1
    print(f"🟢 Đã xử lý {frame_count}/{total_frames} frames", end="\r")

cap.release()
out_video.release()
print(f"\n✅ Video output lưu tại: {video_out_path}")
print(f"📸 Đã lưu {len(saved_ids_card)} người CÓ thẻ vào: {output_dir_card}")
print(f"📸 Đã lưu {len(saved_ids_no_card)} người KHÔNG thẻ vào: {output_dir_no_card}")


Streaming output truncated to the last 5000 lines.
Speed: 3.6ms preprocess, 158.0ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 147.8ms
Speed: 3.8ms preprocess, 147.8ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 146.4ms
Speed: 3.9ms preprocess, 146.4ms inference, 1.1ms postprocess per image at shape (1, 3, 352, 640)
🟢 Đã xử lý 2149/8247 frames
0: 352x640 (no detections), 198.0ms
Speed: 4.9ms preprocess, 198.0ms inference, 1.6ms postprocess per image at shape (1, 3, 352, 640)
🟢 Đã xử lý 2150/8247 frames
0: 352x640 (no detections), 207.8ms
Speed: 5.5ms preprocess, 207.8ms inference, 1.4ms postprocess per image at shape (1, 3, 352, 640)
🟢 Đã xử lý 2151/8247 frames
0: 352x640 (no detections), 222.7ms
Speed: 6.6ms preprocess, 222.7ms inference, 1.3ms postprocess per image at shape (1, 3, 352, 640)
🟢 Đã xử lý 2152/8247 frames
0: 352x640 (no detections), 215.7ms
Speed: 6.9ms preprocess, 215.7m

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import os
import pandas as pd

# Load models
model_person = YOLO('yolov8n.pt')
model_badge = YOLO('/content/drive/MyDrive/Myproject/weight_v1/train/weights/best.pt')

# Thư mục lưu ảnh
wear_dir = "/content/per_wear"
no_wear_dir = "/content/per_no_wear"
os.makedirs(wear_dir, exist_ok=True)
os.makedirs(no_wear_dir, exist_ok=True)

# Video input/output
video_in = "/content/drive/MyDrive/Myproject/ TEST!.mp4"
video_out = "/content/output_.mp4"
cap = cv2.VideoCapture(video_in)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter(video_out, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

# CSV log và ID theo dõi
log = []
saved_ids = set()
frame_idx = 0

def is_centered(inner_box, outer_box, thresh=0.3):
    """Kiểm tra xem bbox nhỏ có nằm gần trung tâm bbox lớn không"""
    ix1, iy1, ix2, iy2 = inner_box
    ox1, oy1, ox2, oy2 = outer_box
    ow, oh = ox2 - ox1, oy2 - oy1
    cx_o, cy_o = ox1 + ow/2, oy1 + oh/2
    cx_i, cy_i = (ix1 + ix2)/2, (iy1 + iy2)/2
    dist = np.sqrt((cx_o - cx_i)**2 + (cy_o - cy_i)**2)
    return dist <= thresh * min(ow, oh)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1
    current_time = round(frame_idx / fps, 2)

    # Detect người có track ID
    results_person = model_person.track(frame, conf=0.4, classes=[0], persist=True, tracker="botsort.yaml")[0]

    # Detect badge/thẻ toàn ảnh
    results_badge = model_badge(frame, conf=0.45)[0]

    # Vẽ tất cả bbox badge
    if results_badge.boxes is not None:
        for box in results_badge.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls_id = int(box.cls[0])
            label = "lanyard" if cls_id == 0 else "card"
            color = (255, 255, 0) if cls_id == 0 else (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Xử lý từng người detect được
    if results_person.boxes is not None:
        for person in results_person.boxes:
            if person.id is None:
                continue
            pid = int(person.id.item())
            x1, y1, x2, y2 = map(int, person.xyxy[0])
            person_crop = frame[y1:y2, x1:x2]

            # Kiểm tra có badge trong vùng người không
            found_valid_badge = False
            if results_badge.boxes is not None:
                for badge in results_badge.boxes:
                    bx1, by1, bx2, by2 = map(int, badge.xyxy[0])
                    if is_centered((bx1, by1, bx2, by2), (x1, y1, x2, y2), thresh=0.3):
                        found_valid_badge = True
                        break

            # Vẽ bbox người
            cv2.rectangle(frame, (x1, y1), (x2, y2), (200, 200, 200), 2)

            # Lưu duy nhất 1 ảnh cho mỗi người
            if pid not in saved_ids:
                saved_ids.add(pid)
                folder = wear_dir if found_valid_badge else no_wear_dir
                path = os.path.join(folder, f"person_{pid}.jpg")
                cv2.imwrite(path, person_crop)
                log.append({
                    "time_sec": current_time,
                    "status": "wear" if found_valid_badge else "no_wear"
                })

    writer.write(frame)
    print(f"🟢 Frame {frame_idx}", end="\r")

# Giải phóng
cap.release()
writer.release()

# Xuất CSV
df = pd.DataFrame(log)
df.to_csv("/content/person_tracking_info.csv", index=False)

print("\n✅ Xử lý hoàn tất.")
print("📁 per_wear:", len(os.listdir(wear_dir)), "ảnh")
print("📁 per_no_wear:", len(os.listdir(no_wear_dir)), "ảnh")
print("📄 CSV tại: /content/person_tracking_info.csv")
print("🎞️ Video tại:", video_out)




Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 143MB/s]


Streaming output truncated to the last 5000 lines.
0: 352x640 1 lanyard, 1 card, 23.7ms
Speed: 3.1ms preprocess, 23.7ms inference, 1.5ms postprocess per image at shape (1, 3, 352, 640)
🟢 Frame 7415
0: 352x640 2 persons, 12.4ms
Speed: 2.9ms preprocess, 12.4ms inference, 2.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 lanyard, 1 card, 23.7ms
Speed: 4.2ms preprocess, 23.7ms inference, 2.2ms postprocess per image at shape (1, 3, 352, 640)
🟢 Frame 7416
0: 352x640 2 persons, 9.0ms
Speed: 2.8ms preprocess, 9.0ms inference, 2.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 lanyard, 1 card, 23.7ms
Speed: 2.9ms preprocess, 23.7ms inference, 1.8ms postprocess per image at shape (1, 3, 352, 640)
🟢 Frame 7417
0: 352x640 2 persons, 10.7ms
Speed: 3.0ms preprocess, 10.7ms inference, 2.1ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 lanyard, 2 cards, 23.8ms
Speed: 3.3ms preprocess, 23.8ms inference, 2.5ms postprocess per image at shape (1, 3, 352,